IMPORTS AND CONFIGURATION

In [1]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy
import diskcache

In [2]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin']
The backtesting folder was not in the systems path
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packag

In [3]:
# from scipy.optimize import minimize
import TradeAndLogics as TL
import Plot
import Data as data
import Data_Processing as dp
from enums import *
from Utility import *

In [4]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\2years'

In [5]:
original_stdout = const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [6]:
index_symbol = 'BANKNIFTY'
start_date = '2023-07-01'
end_date = '2024-11-01'
expiry_type_near = 'I'
expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 15 # mins
look_back_window = 25*4
# Trade/ Strategy Parameters
index_delta_threshold_per_lot = 1
component_delta_threshold_per_lot = 1
notional_vega = 10000 # rs
profit_target = 100000
stop_loss = 100000
zscore_threshold_long = 2
zscore_threshold_short = -2
ic_threshold_long = 0.8
ic_threshold_short = 0.2
epsilon = 0.5

In [7]:
strategy_desc = f"Dispersion_AllComponents_2years_{index_symbol}"

Strategy Specific Data Dealing and Modifications

In [8]:
def insert_index_component(constituents, constituent_symbol, weight, lot_size):
    dict = {
        'weight': weight,
        'lot_size': lot_size
    }
    constituents[constituent_symbol] = dict

In [9]:
constituents_weight_lots = {}

In [10]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

In [11]:
# insert_index_component(constituents_weight_lots, "PNB", 0.91, 8000)
insert_index_component(constituents_weight_lots, "ICICIBANK", 23.03, 700)
# insert_index_component(constituents_weight_lots, "AUBANK", 2.69, 1000)
# insert_index_component(constituents_weight_lots, "BANKBARODA", 1.84, 2925)
# insert_index_component(constituents_weight_lots, "FEDERALBNK", 1.68, 5000)
# insert_index_component(constituents_weight_lots, "IDFCFIRSTB", 1.08, 7500)
insert_index_component(constituents_weight_lots, "SBIN", 11.27, 750)
# insert_index_component(constituents_weight_lots, "INDUSINDBK", 5.58, 500)
insert_index_component(constituents_weight_lots, "HDFCBANK", 27.04, 550)
insert_index_component(constituents_weight_lots, "KOTAKBANK", 11.72, 400)
# insert_index_component(constituents_weight_lots, "BANDHANBNK", 1.98, 2800)
insert_index_component(constituents_weight_lots, "AXISBANK", 11.18, 625)

In [12]:
constituents_weight_lots

{'ICICIBANK': {'weight': 23.03, 'lot_size': 700},
 'SBIN': {'weight': 11.27, 'lot_size': 750},
 'HDFCBANK': {'weight': 27.04, 'lot_size': 550},
 'KOTAKBANK': {'weight': 11.72, 'lot_size': 400},
 'AXISBANK': {'weight': 11.18, 'lot_size': 625}}

In [13]:
def normalize_constituents_weights(constituents_weight_lots):
    weight_sum = 0
    for values in constituents_weight_lots.values():
        weight_sum += values['weight']
    if weight_sum != 100:
        print(f"Sum of weights = {weight_sum} is not 100. Normalizing weights...")
        total_weight = sum([values['weight'] for values in constituents_weight_lots.values()])
        for values in constituents_weight_lots.values():
            values['weight'] = (values['weight'] / total_weight) * 100
    
        normalized_sum = sum([values['weight'] for values in constituents_weight_lots.values()])
        print("Normalized sum:", normalized_sum)
        print(constituents_weight_lots)
    else:
        print(f"provided weights add up to perfect {weight_sum}")
    return (constituents_weight_lots)

In [14]:
normalize_constituents_weights(constituents_weight_lots)

Sum of weights = 84.24000000000001 is not 100. Normalizing weights...
Normalized sum: 100.00000000000001
{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700}, 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750}, 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550}, 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400}, 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}


{'ICICIBANK': {'weight': 27.338556505223178, 'lot_size': 700},
 'SBIN': {'weight': 13.378442545109213, 'lot_size': 750},
 'HDFCBANK': {'weight': 32.098765432098766, 'lot_size': 550},
 'KOTAKBANK': {'weight': 13.912630579297247, 'lot_size': 400},
 'AXISBANK': {'weight': 13.271604938271606, 'lot_size': 625}}

NEAR MONTH (I) DATA FETCHING AND PROCESSING

In [15]:
import Dispersion.DispersionAdjustedFunctionality as daf

In [16]:
constituents_near = {}

In [17]:
logs_near = open(f'LOGS_Fetching_Near_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_near

index_near = daf.ticker(index_symbol, index_lot_size, True, start_date, end_date, expiry_type_near, True, timeframe, False, 0.1)
index_near.initializeDispersion(True, constituents_near, False, 1)
for stock, weight_lots in constituents_weight_lots.items():
    constituents_near[stock] = daf.ticker(stock, weight_lots['lot_size'], True, start_date, end_date, expiry_type_near, True, timeframe, False, 0.1)
    constituents_near[stock].initializeDispersion(False, {}, True, weight_lots['weight']/100)

fix_output(original_stdout, logs_near)

In [18]:
ohlc = OHLC.close

In [19]:
index_near.set_ohlc(ohlc)
for _, constituent in index_near.components.items():
    constituent.set_ohlc(ohlc)

OHLC for BANKNIFTY is set to OHLC.close
OHLC for ICICIBANK is set to OHLC.close
OHLC for SBIN is set to OHLC.close
OHLC for HDFCBANK is set to OHLC.close
OHLC for KOTAKBANK is set to OHLC.close
OHLC for AXISBANK is set to OHLC.close


In [20]:
# logs_bhav_near = open(f'LOGS_Fetching_BHAV_Data_for_Near_Month.txt', 'w')
# sys.stdout = logs_bhav_near
# index_near.get_bhav_futures_data()
# for _, component in index_near.components.items():
#     component.get_bhav_futures_data()
# sys.stdout = original_stdout.value
# logs_bhav_near.close()

NEXT MONTH (II) DATA FETCHING AND PROCESSING

In [21]:
constituents_next = {}

In [22]:
logs_next = open(f'LOGS_Fetching_Next_Month_Data_for_{strategy_desc}.txt', 'w')
sys.stdout = logs_next

index_next = daf.ticker(index_symbol, index_lot_size, True, start_date, end_date, expiry_type_next, False, timeframe, False, 0.1)
index_next.initializeDispersion(True, constituents_next, False, 1)
for stock, weight_lots in constituents_weight_lots.items():
    constituents_next[stock] = daf.ticker(stock, weight_lots['lot_size']/100, True, start_date, end_date, expiry_type_next, False, timeframe, False, 0.1)
    constituents_next[stock].initializeDispersion(False, {}, True, weight_lots['weight'])

fix_output(original_stdout, logs_next)

In [23]:
ohlc = OHLC.close

In [24]:
index_next.set_ohlc(ohlc)
for _, constituent in index_next.components.items():
    constituent.set_ohlc(ohlc)

OHLC for BANKNIFTY is set to OHLC.close
OHLC for ICICIBANK is set to OHLC.close
OHLC for SBIN is set to OHLC.close
OHLC for HDFCBANK is set to OHLC.close
OHLC for KOTAKBANK is set to OHLC.close
OHLC for AXISBANK is set to OHLC.close


BEGINS

In [25]:
def get_index_and_components_iv(index):
    index.generate_iv_data()
    for _, component in index.components.items():
        component.generate_iv_data()

In [26]:
logs_iv_near = open(f'LOGS_IV_Generation_for_Near_Month_{strategy_desc}.txt', 'w')
sys.stdout = logs_iv_near
get_index_and_components_iv(index_near)
sys.stdout = original_stdout.value
logs_iv_near.close()

In [27]:
logs_iv_next = open(f'LOGS_IV_Generation_for_Next_Month_{strategy_desc}.txt', 'w')
sys.stdout = logs_iv_next
get_index_and_components_iv(index_next)
sys.stdout = original_stdout.value
logs_iv_next.close()

NORMALIZE HORIZONTALLY OVER HERE

In [28]:
def get_normallized_iv(timestamp, ticker_near, ticker_next):
    print(f"  {ticker_near.symbol}")
    iv_near = ticker_near.df_futures.loc[timestamp, 'iv']
    iv_next = ticker_next.df_futures.loc[timestamp, 'iv']
    print(f">>  IV Near: {iv_near} || IV Next: {iv_next}")
    expiry_near = pd.to_datetime(ticker_near.df_futures.loc[timestamp, 'expiry'])
    expiry_next = pd.to_datetime(ticker_next.df_futures.loc[timestamp, 'expiry'])
    print(f">>  Expiry Near: {expiry_near.strftime('%d/%b/%Y')} || Expiry Next: {expiry_next.strftime('%d/%b/%Y')}")
    time_to_expiry_near = (expiry_near - timestamp).days
    time_to_expiry_next = (expiry_next - timestamp).days
    print(f">>  Time to Expiry Near: {time_to_expiry_near}days || Time to Expiry Next: {time_to_expiry_next}days")
    if pd.isna(iv_near):
        # print(f"iv_near = {iv_near} | pd.isna(iv_near) = {pd.isna(iv_near)}")
        print(f"Invalidating timestamp because of NA IV's.")
        return pd.NA
    weight1 = (time_to_expiry_next - 30)/(time_to_expiry_next - time_to_expiry_near) if not pd.isna(iv_next) else 1
    weight2 = 1 - weight1
    print(f">>  Weight1 Near: {weight1} || Weight2 Next: {weight2}")
    normalized_iv = weight1 * iv_near 
    if not pd.isna(iv_next):
        normalized_iv += + weight2 * iv_next
    print(f">>  Normallized IV = {normalized_iv}")
    return normalized_iv

In [29]:
def get_index_with_normalized_iv(index_near, index_next):
    logs_normalization_iv = open(f'LOGS_IV_Normalization_for_{strategy_desc}.txt', 'w')
    sys.stdout = logs_normalization_iv
    index_fresh = copy.deepcopy(index_near)
    print("Normallizing IV horizontally using Near and Next month options")
    for timestamp in index_near.df_futures.index:
        print(f"Timestamp: {timestamp}")
        index_fresh.df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, index_near, index_next)
        for component_near, component_next in zip(index_near.components.values(), index_next.components.values()):
            index_fresh.components[component_near.symbol].df_futures.loc[timestamp, 'iv'] = get_normallized_iv(timestamp, component_near, component_next)
        print('-------------------------------------------------------------')
    sys.stdout = original_stdout.value
    logs_normalization_iv.close()
    return index_fresh

In [30]:
Index = get_index_with_normalized_iv(index_near, index_next)

KeyError: Timestamp('2024-08-30 09:15:00')

In [31]:
index = copy.deepcopy(Index)

BACK TO NORMAL

In [32]:
index_near.df_futures['iv']

2023-07-03 09:15:00    0.124684
2023-07-03 09:30:00    0.131162
2023-07-03 09:45:00    0.133409
2023-07-03 10:00:00    0.131409
2023-07-03 10:15:00    0.131087
                         ...   
2023-08-01 14:15:00      0.1147
2023-08-01 14:30:00    0.114779
2023-08-01 14:45:00    0.113853
2023-08-01 15:00:00    0.114086
2023-08-01 15:15:00    0.113518
Name: iv, Length: 550, dtype: object

In [33]:
index_next.df_futures['iv']

2023-07-03 09:15:00        <NA>
2023-07-03 09:30:00    0.126907
2023-07-03 09:45:00    0.127961
2023-07-03 10:00:00    0.128212
2023-07-03 10:15:00    0.127539
                         ...   
2023-08-01 14:15:00    0.118847
2023-08-01 14:30:00    0.119548
2023-08-01 14:45:00    0.119464
2023-08-01 15:00:00    0.119363
2023-08-01 15:15:00    0.119295
Name: iv, Length: 550, dtype: object

In [34]:
index.df_futures['iv']

2023-07-03 09:15:00    0.124684
2023-07-03 09:30:00    0.130432
2023-07-03 09:45:00    0.132475
2023-07-03 10:00:00    0.130861
2023-07-03 10:15:00    0.130479
                         ...   
2023-08-01 14:15:00      0.1147
2023-08-01 14:30:00    0.114779
2023-08-01 14:45:00    0.114054
2023-08-01 15:00:00    0.114274
2023-08-01 15:15:00    0.113724
Name: iv, Length: 550, dtype: object

In [35]:
logs_ic = open(f'LOGS_IC_Generation_for{strategy_desc}.txt', 'w')
sys.stdout = logs_ic
index.generate_ic_data()
sys.stdout = original_stdout.value
logs_ic.close()

In [ ]:
index.df_futures['filled_ic'] = index.df_futures['ic'].ffill()
# index.df_futures['ic'] = index.df_futures['ic'].replace({pd.NA: np.nan})
window = 3000
rolling_mean = index.df_futures['filled_ic'].rolling(window=window, min_periods=window).mean()
rolling_std = index.df_futures['filled_ic'].rolling(window=window, min_periods=window).std()
index.df_futures['zscore'] = (index.df_futures['filled_ic'] - rolling_mean) / rolling_std


In [ ]:
rolling_std

In [ ]:
zscore = index.df_futures['zscore']
ddf = pd.DataFrame({'zscore_smoothened': zscore.ewm(span=5).mean(), 'zscore': zscore, 'ema long': zscore.ewm(span=100).mean(), 'ema short': zscore.ewm(span=40).mean()})
smooth_zscore = Plot.plot_df(ddf, 'zscore_smoothened', 'zscore', 'ema long', 'ema short')
smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, -2, 0, len(ddf), 'Red', 1)
smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, zscore_threshold_short, 0, len(ddf), 'Blue', 1)
smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, zscore_threshold_long, 0, len(ddf), 'Blue', 1)
smooth_zscore = Plot.draw_horizontal_line(smooth_zscore, 2, 0, len(ddf), 'Red', 1)
smooth_zscore.show()
Plot.save_plot(smooth_zscore, f'PLOT_smoothened_zscore_for_{strategy_desc}.html')

In [69]:
iv_df = []
iv_df.append(index.df_futures['iv'].rename(index.symbol))
for component in index.components.values():
    iv_df.append(component.df_futures['iv'].rename(component.symbol))
iv_df = pd.concat(iv_df, axis=1)
fig_iv = Plot.plot_df(iv_df*100, *iv_df.columns)
Plot.save_plot(fig_iv, f'PLOT_Normalized_IV_for_{strategy_desc}.html')
fig_iv.show()

In [ ]:
fig_ic = Plot.plot_df(index.df_futures, 'filled_ic')
fig_ic.show()
Plot.save_plot(fig_ic, f'PLOT_ic_for_{strategy_desc}.html')

In [ ]:
fig_zscore = Plot.plot_df(index.df_futures, 'zscore')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, -2, 0, len(index.df_futures), 'Red')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_short, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, zscore_threshold_long, 0, len(index.df_futures), 'Blue')
fig_zscore = Plot.draw_horizontal_line(fig_zscore, 2, 0, len(index.df_futures), 'Red')
fig_zscore.show()
Plot.save_plot(fig_zscore, f'PLOT_zscore_for_{strategy_desc}.html')

In [ ]:
x = index.df_futures['zscore']
count = 0
flag = False
for z in x:
    if z > zscore_threshold_long and flag == False:
        flag = True
        count+=1
    if z < zscore_threshold_short and flag == True:
        flag = False
        count += 1
print("theoretically number of trades just with zscore signals", count)

In [43]:
positions = {}

In [44]:
def get_straddle_vega(ticker, timestamp):
    vega = 0
    for option_type in Option:
        strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        greeks = ticker.greeks(timestamp, option_type, strike)
        if greeks is None:
            print(f"get_straddle_vega: greeks is None || Symbol: {ticker.symbol} | OHLC: {ticker.ohlc.name} | Spot: {ticker.get_futures_price(timestamp)} | Strike: {strike} | Opt Price: {ticker.get_opts(option_type).loc[timestamp, strike]} | T(days): {pd.to_datetime(ticker.get_opts(option_type).loc[timestamp, 'expiry']).date() - pd.to_datetime(timestamp).date()} | Option: {option_type.name}")
            return None
        vega += greeks['vega']
    return vega

In [45]:
def get_lots_for_neutrality(logging_information, vegas, weights, notional_vega):    
    lots_for_neutrality = []
    for symbol, _ in vegas.items():
        lots = notional_vega * weights[symbol] / vegas[symbol]
        lots_for_neutrality.append(lots)
        logging_information[symbol]['Lots to take position'] = int(lots)
    return int(np.round(lots_for_neutrality[0])), np.round(lots_for_neutrality[1:]).astype(int)

In [46]:
def get_proportionality_lots(logging_information, index_value, values, weights, notional_scaling_value):
    lots_components = {}
    lots_index = 1
    logging_information[index.symbol]['Lots to take position'] = 1
    for key, weight in weights.items():
        lots_components[key] = (index_value * weight) / values[key]
        logging_information[key]['Lots to take position'] = int(np.round((index_value * weight) / values[key]))
    if notional_scaling_value is not None:
        lots_index = notional_scaling_value/index_value
        logging_information[index.symbol]['Lots to take position'] = int(np.round(lots_index))
        for key, value in lots_components.items():
            lots_components[key] = value * lots_index
            logging_information[key]['Lots to take position'] = int(np.round(value * lots_index))
    # return lots_index, [value for key, value in lots_components.items()]
    return int(np.round(lots_index)), [int(np.round(value)) for key, value in lots_components.items()]

In [47]:
def entry_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore']
    ic = index.df_futures.loc[timestamp, 'ic']
    return zscore >= zscore_threshold_long
    # return ic >= ic_threshold_long


def exit_signal(timestamp):
    zscore = index.df_futures.loc[timestamp, 'zscore'] 
    ic = index.df_futures.loc[timestamp, 'ic'] 
    return zscore <= zscore_threshold_short
    # return ic <= ic_threshold_short

def get_lots_for_dispersion(dispersion_position_of_index, index, timestamp, net_vega_neutral = False, vega_neutral = False, strike_neutral = False, theta_neutral = False, notional_vega = 10000):
    # If you want to do vega neutral entry
    unweighted_vegas, weights, logging_information = {}, {}, {}
    vega_index_atm_straddle = get_straddle_vega(index, timestamp)
    if vega_index_atm_straddle is None:
        return None, None
    unweighted_vegas[index.symbol] = vega_index_atm_straddle * index.lot_size
    weights[index.symbol] = 1
    strike_index, _ = index.find_moneyness_strike(timestamp, 0, Option.Call)
    logging_information[index.symbol] = {
        'Lot Size': index.lot_size,
        'Weight': index.weight,
        'Vega per Straddle': vega_index_atm_straddle,
        'Vega per Lot Straddle': vega_index_atm_straddle * index.lot_size,
    }
    if net_vega_neutral:
        logging_information[index.symbol]['Target Notional Vega'] = notional_vega
    if vega_neutral:
        logging_information[index.symbol]['Target Notional Vega'] = notional_vega
    if theta_neutral:
        logging_information[index.symbol]['ATM Strike'] = strike_index

    for _, component in index.components.items():
        vega_component_atm_straddle = get_straddle_vega(component, timestamp)
        if vega_component_atm_straddle is None:
            return None, None
        unweighted_vegas[component.symbol] = vega_component_atm_straddle * component.lot_size
        weights[component.symbol] = component.weight 
        strike_component, _ = component.find_moneyness_strike(timestamp, 0, Option.Call)
        logging_information[component.symbol] = {
            'Lot Size': component.lot_size,
            'Weight': component.weight,
            'Vega per Straddle': vega_component_atm_straddle,
            'Vega per Lot Straddle': vega_component_atm_straddle * component.lot_size
        }
        if net_vega_neutral:
            logging_information[component.symbol]['Target Notional Vega'] = notional_vega * component.weight
        if vega_neutral:
            logging_information[component.symbol]['Target Notional Vega'] = notional_vega * component.weight
        if theta_neutral:
            logging_information[component.symbol]['ATM Strike'] = strike_component

    if net_vega_neutral:
        lots_index, lots_components = get_lots_for_neutrality(logging_information, unweighted_vegas, weights, notional_vega)

    if vega_neutral: 
        lots_index, lots_components = get_proportionality_lots(
                logging_information, 
                unweighted_vegas[index.symbol], 
                {k: v for k, v in unweighted_vegas.items() if k != index.symbol}, 
                {k: v for k, v in weights.items() if k != index.symbol}, 
                notional_vega
        )
    # If you want to do a strike neutral entry
    if strike_neutral:

        pass
    # If you want to do a theta/ correlation neutral entry
    if theta_neutral:
            summation = sum([component.weight * logging_information[symbol]['ATM Strike'] for symbol, component in index.components.items()])
            sqrt_p = strike_index/summation
            print(strike_index, summation, sqrt_p)
            import math
            lots_index, lots_components = get_proportionality_lots(
                logging_information, 
                unweighted_vegas[index.symbol], 
                {k: v for k, v in unweighted_vegas.items() if k != index.symbol},
                {k: v*math.sqrt(index.df_futures.loc[timestamp, 'filled_ic']) for k, v in weights.items() if k != index.symbol}, 
                notional_vega
            )
            
    for symbol, symbol_information in logging_information.items():
        print(f"{symbol} || Lots: {symbol_information['Lots to take position']}")
        for info_key, info_value in symbol_information.items():
            # if(info_key == "Lots to take position"):
            #     continue
            print(f"  {info_key}: {info_value}")

    return lots_index, lots_components


In [48]:
# timestamp = pd.to_datetime('2023-07-03 09:45:00')

In [49]:
# get_lots_for_dispersion(None, index, timestamp, False, False, False, True, 100000)
# 72, [41, 644, 67, 61, 49, 34, 482, 151, 522, 293, 55, 336])

In [50]:
# get_lots_for_dispersion(None, index, timestamp, True, False, False, False, 100000)
# (72, array([ 1, 17,  2,  2,  1,  1, 13,  4, 14,  8,  1,  9]))

In [51]:
# get_lots_for_dispersion(None, index, timestamp, False, True, False, False, 100000)
# (72, array([ 1, 17,  2,  2,  1,  1, 13,  4, 14,  8,  1,  9]))

In [52]:
def get_net_delta(ticker, timestamp):
    net_delta = 0
    for _, token in ticker.tokens.items():
        net_delta += token.stats.loc[timestamp, 'net_delta']
    return net_delta

In [53]:
def take_dispersion_position(time_data, remarks, ticker, position, lots):
    timestamp, time_ix = time_data
    legs = []
    for option_type in Option:
        atm_strike, _ = ticker.find_moneyness_strike(timestamp, 0, option_type)
        key = f'{ticker.symbol}_{option_type.name}_{atm_strike}' 
        if key not in positions:
            positions[key] = TL.Token(ticker, option_type, atm_strike, FNO.OPTIONS)
        token = positions[key]
        # token.update_df(time_ix, timestamp)
        token.add_position(timestamp, lots, position)
        ticker.update_token(key, token)
        leg = {
            "position" : position, 
            "opt_type" : option_type, 
            "strike": atm_strike, 
            "lots": lots
        }
        legs.append(leg)
    ticker.take_position(timestamp, remarks, *legs)


In [54]:
def hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1, logging = True):
    print(f"-------  Hedging Check for {ticker.symbol}  -----------")
    net_delta = get_net_delta(ticker, timestamp)
    delta_threshold_per_lot = index_delta_threshold_per_lot if ticker.is_index else component_delta_threshold_per_lot 
    print(f"Notional Delta: {round(net_delta)} | (Net Delta)/(Lot Size of {ticker.symbol}): {np.round(abs(net_delta)/ticker.lot_size, 5)} | Threshold Lots: {delta_threshold_per_lot}")
    if abs(net_delta) >= delta_threshold_per_lot * ticker.lot_size:
        qty_futures = (-net_delta)/ticker.lot_size
        position = LongShort(np.sign(qty_futures))
        lots_futures = int(abs(qty_futures))
        if lots_futures != 0:
            # ticker.hedge_futures_trade(lots_futures, position, timestamp)
            key = f'{ticker.symbol}_Futures'
            if key not in positions:
                positions[key] = TL.Token(ticker, None, None, FNO.FUTURES)
            token = positions[key]
            # token.update_df(time_ix, timestamp, logging)
            token.add_position(timestamp, lots_futures, position)
            ticker.update_token(key, token) #python obj ref prop, check
            token.ticker = ticker
            positions[key] = token #python obj ref prop, check
            leg = {
                "position" : position, 
                "opt_type" : FNO.FUTURES, 
                "strike": None,
                "lots": lots_futures
            }
            ticker.take_position(timestamp, remarks, leg)
    
            print(f"Added {lots_futures * position.value * ticker.lot_size} notional delta by going {position.name} {lots_futures} lots of future. new delta = {round(net_delta) + lots_futures * position.value * ticker.lot_size}")
            print(f"-------  Hedged Delta {ticker.symbol}  -------")
        else:
            print(f"-------  Hedging condition threshold per option = {delta_threshold_per_lot} for {ticker.symbol} met but required lots_futures were 0 so did not hedge  -------")
    else:
        print(f"-------  Did not Hedge {ticker.symbol}  -------")

In [55]:
def reset_hedging_positions(ticker):
    ticker.hedging = pd.DataFrame(columns = ['date_timestamp', 'lots', 'position', 'futures_price', 'delta_added'])
    ticker.hedging.set_index('date_timestamp', inplace=True)

def reset_trade_positions(ticker):
    ticker.Trades = dp.Trades()
    ticker.tokens = {}

In [57]:
def HandleUpdate(ticker, time_ix, timestamp, remarks, hedge = True, logging_token_update=True, logging_hedging=True, index_delta_threshold_per_lot = 5, component_delta_threshold_per_lot = 1):
    print()
    print(f">> {ticker.symbol}")
    active_tokens = {}
    inactive_tokens = {}
    for key, token in ticker.tokens.items():
        if token.position.value != 0:
            active_tokens[key] = token
        else:
            inactive_tokens[key] = token
    print(f"  ----- Active Tokens -----")
    for key, token in active_tokens.items():
        token.update_df(time_ix, timestamp, logging_token_update)
        positions[key] = token #python obj ref prop, check
        if not logging_hedging:
            continue
        if token.fno == FNO.OPTIONS:
            print(f"{token.ticker.symbol, token.opt_type.name, float(token.strike)} | position = {token.position.name} | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} (position accounted) | Lot size: {ticker.lot_size} | Lots in position: {token.lots}")
        else:
            print(f"{token.ticker.symbol, token.fno.name}  | position = {token.position.name} | Notional Delta = {np.round(token.stats.loc[timestamp, 'net_delta'], 2)} | Lot size: {ticker.lot_size} | Lots in position: {token.lots}")
    print(f"  ----- Inactive Tokens -----")
    for key, token in inactive_tokens.items():
        token.update_df(time_ix, timestamp, False)
        positions[key] = token #python obj ref prop, check
        if not logging_hedging:
            continue
        if token.fno == FNO.OPTIONS:
            print(f"{token.ticker.symbol, token.opt_type.name, float(token.strike)} | position = {token.position.name}")
        else:
            print(f"{token.ticker.symbol, token.fno.name} | position = {token.position.name}")
    
    if hedge:
        if not TL.check_existing_position(ticker):
            return
        hedge_delta(time_ix, timestamp, remarks, ticker, index_delta_threshold_per_lot, component_delta_threshold_per_lot)
    for key, token in ticker.tokens.items():
        token.ticker = ticker
        positions[key] = token #python obj ref prop, check

In [58]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        # return True
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        index = self.tickers[0]
        if self.triggered_at is None:
            return
        for z in index.df_futures['zscore'].loc[self.triggered_at : timestamp]:
            if abs(z) <= epsilon:
                self.triggered_at = None
                self.active_ptsl = False  
                print(f"PTSL reverted back to mean (<= {epsilon}): Future Signals can now be used for trading")

In [59]:
def squareoff_ticker(time_ix, timestamp, remarks, ticker, logging):
    legs_trade, leg_hedge = [], []
    if(not TL.check_existing_position(ticker)):
        print(f">> {ticker.symbol} does not hold any position")
        print()
        return
    print(f">> Squaring off {ticker.symbol}")
    for key, token in ticker.tokens.items():
        token.update_df(time_ix, timestamp, logging)
        if token.position == LongShort.Neutral:
            continue
        position_to_take = LongShort(-token.position.value) 
        if token.fno == FNO.OPTIONS:
            leg = {
            "position" : position_to_take, 
            "opt_type" : token.opt_type, 
            "strike": token.strike,
            "lots": token.lots
            }
            legs_trade.append(leg)
        else:
            leg = {
            "position" : position_to_take, 
            "opt_type" : FNO.FUTURES, 
            "strike": token.strike,
            "lots": token.lots
            }
            leg_hedge.append(leg)
        token.add_position(timestamp, token.lots, position_to_take)
        positions[key] = token #python obj ref prop, check
        ticker.tokens[key] = token #python obj ref prop, check
    if len(legs_trade):
        ticker.take_position(timestamp, remarks, *legs_trade)
    else:    
        print(f" Did not find any trading legs for {ticker.symbol}")
    if len(leg_hedge):
        ticker.take_position(timestamp, remarks, *leg_hedge)
    else:
        print(f" Did not find any hedging legs for {ticker.symbol}")
        
    allSecurities = leg_hedge + legs_trade
    if len(allSecurities):
        # secDesc = [f'{ticker.get_expiry(timestamp).strftime('%d%b')}|{int(leg['strike']) if leg['strike'] is not None else ""}|{leg['opt_type'].name}' for leg in allSecurities]
        for leg in allSecurities:
            if leg['opt_type'] == FNO.FUTURES:
                print(f"  Squared off {ticker.symbol, leg['opt_type'].name}")
            else:
                print(f"  Squared off {ticker.symbol, leg['opt_type'].name, int(leg['strike'])}")

        print()

    # Every Token in a Ticker should have the updated Ticker in them
    for key, token in ticker.tokens.items():
        token.ticker = ticker #python obj ref prop, check
        positions[key] = token #python obj ref prop, check

In [60]:
def squareoff(time_ix, timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    # for key, token in positions.items():
    #     token.add_position(token.lots, LongShort(-token.position.value))
    # print("******  positions added in GLOBAL TOKENS. Now updating the key-information(running_pnl, lots, !(delta, vega, iv)) in them  ******")
    # for key, token in positions.items():
    #     token.update_df(time_ix, timestamp, logging)
    #     positions[key] = token
    # print("******  Information updated globally. TICKER WISE UPDATION BEGINS  ******")
    squareoff_ticker(time_ix, timestamp, remarks, index, logging)
    for component in index.components.values():
        squareoff_ticker(time_ix, timestamp, remarks, component, logging)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [61]:
def isTodayAnyExpiry(timestamp, index):
    is_expiry = False
    is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(index.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    for component in index.components.values():
        is_expiry = is_expiry or (pd.to_datetime(timestamp).date() == pd.to_datetime(component.get_opts(Option.Call).loc[timestamp, 'expiry']).date())
    return is_expiry

In [62]:
def UpdateDispersionTickers(time_ix, timestamp, remarks, hedge, logging_token_update, logging_hedging):
    HandleUpdate(index, time_ix, timestamp, remarks, 
                 hedge, logging_token_update, logging_hedging, 
                 index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                 component_delta_threshold_per_lot=component_delta_threshold_per_lot)
    for component in index.components.values():
        HandleUpdate(component, time_ix, timestamp, remarks, 
                     hedge, logging_token_update, logging_hedging, 
                     index_delta_threshold_per_lot=index_delta_threshold_per_lot, 
                     component_delta_threshold_per_lot=component_delta_threshold_per_lot)

In [ ]:
# TL.zoom_tokens_performance_bar_by_bar(index)['Combined Summary']

In [ ]:
def zoom_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    performance = TL.zoom_tokens_performance_bar_by_bar(*portfolio, start=start, end=end)

    for name, df in performance.items():
        if name != "Combined Summary":
            df['Z-Score'] = index.df_futures['zscore'].loc[start:end]
            df['IC'] = index.df_futures['ic'].loc[start:end]

    Plot.save_df_to_excel(performance, file_name)

In [63]:
# def zoom_trade(start, end, output_filename="zoom_trade_output.xlsx"):
#     import pandas as pd
#     from openpyxl import Workbook
#     from openpyxl.utils.dataframe import dataframe_to_rows
#     from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
#     from openpyxl.worksheet.table import Table, TableStyleInfo
#     start = pd.to_datetime(start)
#     end = pd.to_datetime(end)
#     with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
#         # Loop through each symbol and create a DataFrame for each
#         symbols = [index.symbol] + [symbol for symbol in index.components.keys()]
#         for symbol in symbols:
#             # timestamps = index.df_futures.loc[start:end].index
#             # df = pd.DataFrame(0.0, index=timestamps, columns=[
#             #     'Call Price', 'Call Strike', 'Call Lots', 'Call Position', 'Call Delta (L)', 'Notional Call Delta', 
#             #     'Put Price', 'Put Strike', 'Put Lots', 'Put Position', 'Put Delta (L)', 'Notional Put Delta',
#             #     'Futures Price', 'Futures Lots', 'Futures Position', 'Net Delta', 
#             #     'Underlying IV', 'Index IV', 'IC', 
#             #     'Stock Running PNL (without expenses)', 'Cummulative Expenses', 'Stock Running PNL (with expenses)'
#             # ])
#             # df['Futures Position'] = df['Futures Position'].astype(object)
#             # df['Call Position'] = df['Call Position'].astype(object)
#             # df['Put Position'] = df['Put Position'].astype(object)

#             # tokens = [token for token in positions.values() if token.ticker.symbol == symbol]
#             # for timestamp in timestamps:
#             #     for token in tokens:
#             #         if token.fno != FNO.FUTURES and (token.stats.loc[timestamp, 'position'] == LongShort.Neutral or token.stats.loc[timestamp, 'position'] == 0):
#             #             continue
#             #         df.loc[timestamp, 'Stock Running PNL (without expenses)'] += token.stats.loc[timestamp, 'running_pnl_without_expenses']
#             #         df.loc[timestamp, 'Cummulative Expenses'] += token.stats.loc[timestamp, 'expenses']
#             #         df.loc[timestamp, 'Stock Running PNL (with expenses)'] += token.stats.loc[timestamp, 'running_pnl']
#             #         df.loc[timestamp, 'Net Delta'] += token.stats.loc[timestamp, 'net_delta']
#             #         df.loc[timestamp, 'Underlying IV'] = token.ticker.df_futures.loc[timestamp, 'iv'] * 100
#             #         df.loc[timestamp, 'Index IV'] = index.df_futures.loc[timestamp, 'iv'] * 100
#             #         df.loc[timestamp, 'IC'] = index.df_futures.loc[timestamp, 'ic']
#             #         if token.fno == FNO.FUTURES:
#             #             df.loc[timestamp, 'Futures Price'] = token.data.loc[timestamp]
#             #             df.loc[timestamp, 'Futures Lots'] = token.stats.loc[timestamp, 'net_lots']
#             #             df.loc[timestamp, 'Futures Position'] = token.stats.loc[timestamp, 'position']
#             #         elif token.opt_type == Option.Call:
#             #             df.loc[timestamp, 'Call Price'] = token.data.loc[timestamp]
#             #             df.loc[timestamp, 'Call Strike'] = token.strike
#             #             df.loc[timestamp, 'Call Lots'] = token.stats.loc[timestamp, 'net_lots']
#             #             df.loc[timestamp, 'Call Delta (L)'] = token.stats.loc[timestamp, 'net_delta']/(token.lot_size) 
#             #             df.loc[timestamp, 'Notional Call Delta'] = token.stats.loc[timestamp, 'net_delta']
#             #             df.loc[timestamp, 'Call Position'] = token.stats.loc[timestamp, 'position']
#             #         elif token.opt_type == Option.Put:
#             #             df.loc[timestamp, 'Put Price'] = token.data.loc[timestamp]
#             #             df.loc[timestamp, 'Put Strike'] = token.strike
#             #             df.loc[timestamp, 'Put Lots'] = token.stats.loc[timestamp, 'net_lots']
#             #             df.loc[timestamp, 'Put Delta (L)'] = token.stats.loc[timestamp, 'net_delta']/(token.lot_size)
#             #             df.loc[timestamp, 'Notional Put Delta'] = token.stats.loc[timestamp, 'net_delta']
#             #             df.loc[timestamp, 'Put Position'] = token.stats.loc[timestamp, 'position']
            
#             # Write each symbol's DataFrame to a separate sheet in the Excel file
#             sheet_name = symbol
#             df.to_excel(writer, sheet_name=sheet_name, index=True)
            
#             # Access the sheet to apply formatting
#             workbook = writer.book
#             worksheet = workbook[sheet_name]
            
#             # Set bold font and alignment for the header row
#             for cell in worksheet["1:1"]:
#                 cell.font = Font(bold=True)
#                 cell.alignment = Alignment(horizontal="center", vertical="center")
#             worksheet.row_dimensions[1].height = 20

#             # Define styles for each column group
#             call_fill = PatternFill(start_color="CCE5FF", end_color="CCE5FF", fill_type="solid")  # Light blue for Call columns
#             put_fill = PatternFill(start_color="FFCCCC", end_color="FFCCCC", fill_type="solid")  # Light red for Put columns
#             futures_fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")  # Light green for Futures columns
#             others_fill = PatternFill(start_color="FFFFCC", end_color="FFFFCC", fill_type="solid")  # Light yellow for other columns

#             # Define a thin border for better visibility
#             thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))

#             # Apply styles by column group
#             call_columns = ['B', 'C', 'D', 'E']  # Call columns
#             put_columns = ['F', 'G', 'H', 'I']   # Put columns
#             futures_columns = ['J', 'K']         # Futures columns
#             others_columns = ['L', 'M', 'N', 'O']  # Other columns

#             for col_letter in call_columns:
#                 for cell in worksheet[col_letter]:
#                     cell.fill = call_fill
#                     cell.border = thin_border

#             for col_letter in put_columns:
#                 for cell in worksheet[col_letter]:
#                     cell.fill = put_fill
#                     cell.border = thin_border

#             for col_letter in futures_columns:
#                 for cell in worksheet[col_letter]:
#                     cell.fill = futures_fill
#                     cell.border = thin_border

#             for col_letter in others_columns:
#                 for cell in worksheet[col_letter]:
#                     cell.fill = others_fill
#                     cell.border = thin_border

#             # Adjust column width for readability
#             for column in worksheet.columns:
#                 max_length = max(len(str(cell.value)) for cell in column)
#                 worksheet.column_dimensions[column[0].column_letter].width = max_length + 2
            
#             # Apply alternating row colors for better visibility
#             for row in worksheet.iter_rows(min_row=2, max_row=worksheet.max_row, min_col=1, max_col=worksheet.max_column):
#                 for cell in row:
#                     if cell.row % 2 == 0:
#                         cell.fill = PatternFill(start_color="F6F6F6", end_color="F6F6F6", fill_type="solid")
#                     else:
#                         cell.fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
            
#             # Apply table style
#             tab = Table(displayName=f"Table_{symbol}", ref=worksheet.dimensions)
#             style = TableStyleInfo(
#                 name="TableStyleMedium9",
#                 showFirstColumn=False,
#                 showLastColumn=False,
#                 showRowStripes=True,
#                 showColumnStripes=True
#             )
#             tab.tableStyleInfo = style
#             worksheet.add_table(tab)
    
#     print(f"Excel file '{output_filename}' created with separate sheets for each symbol.")


In [174]:
def visualise_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    portfolio = [index, *[component for component in index.components.values()]]
    result_df = TL.get_summary_portfolio(*portfolio, start=start, end=end)

    result_df['Z-Score'] = index.df_futures['zscore'].loc[start:end]
    result_df['IC'] = index.df_futures['ic'].loc[start:end]

    fig = Plot.plot_df(result_df, *(result_df.columns))
    Plot.save_plot(fig, file_name)

In [ ]:
positions = {}
reset_hedging_positions(index)
reset_trade_positions(index)
for component in index.components.values():
    reset_hedging_positions(component)
    reset_trade_positions(component)


PTSL = DispersionPTSL(profit_target, stop_loss, index, *index.components.values())
current_position = None

file = open(f'LOGS_Trading_Logic_for_{strategy_desc}.txt', 'w')
sys.stdout = file
trade_start_date = None
trade_end_date = None
trade_count = 0

for time_ix, timestamp in enumerate(index.df_futures.index):
    print()
    print(f"Timestamp: {timestamp}")
    print(f"IC = {index.df_futures.loc[timestamp, 'ic']} | Z-Score = {index.df_futures.loc[timestamp, 'zscore']} | Filled IC = {index.df_futures.loc[timestamp, 'filled_ic']}")
    #######################################################################################################################################################
    PTSL.update_validity(timestamp)
    if TL.check_existing_position(index):
        
        print(f"Existing Position Check: TRUE | Dispersion position: {current_position}")
        
        # EXPIRY DAY MANUAL SQUAREOFF
        if isTodayAnyExpiry(timestamp, index):
            print("************  EXPIRY DAY, MANUAL SQUARE OFF ************  ")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, False, False)
            squareoff(time_ix, timestamp, 'Expiry Day, Manual Square Off', False)
            trade_end_date = index.df_futures.index[time_ix+1]
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
            continue
            
            
        # PROFIT TARGET AND STOP LOSS SQUAREOFF
        ptsl_result = PTSL.status(timestamp) 
        if ptsl_result != "good_to_go":
            print("************  PTSL Trigger hit *********** ")
            print(f"{PTSL.nature} Square off at NetPnl of {PTSL.pnl_last_trade}")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, f'{ptsl_result} Square Off', True)
            trade_end_date = index.df_futures.index[time_ix+1]
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
            continue
            
            
        # REVERSE SIGNAL SQUAREOFF
        squareoff_signal = (current_position == LongShort.Long and exit_signal(timestamp)) or (current_position == LongShort.Short and entry_signal(timestamp))
            
        if squareoff_signal:
            print("Entered squareoff signal block")
            print("************  SquareOff Reverse zscore Signal hit. ************ ")
            UpdateDispersionTickers(time_ix, timestamp, 'NoHedging', False, True, False)
            squareoff(time_ix, timestamp, 'ZScore Reversal Square Off (Entry in next timestamp/ signal)')
            trade_end_date = index.df_futures.index[time_ix+1]
            trade_count+=1
            zoom_dispersion_trade(trade_start_date, trade_end_date, f"Trade_{trade_count}.xlsx")
            visualise_dispersion_trade(trade_start_date, trade_end_date, f"Visualise_Trade_{trade_count}.html")
            continue
            
        # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
        UpdateDispersionTickers(time_ix, timestamp, 'Delta Hedging', True, True, True)
        print("========================================================================================================================================================================")
        continue
    #######################################################################################################################################################
    
    
    #######################################################################################################################################################
    if entry_signal(timestamp):
        print("************  LONG DISPERSION SIGNAL ************  ")
        UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            continue

        if not PTSL.is_valid(timestamp):
            print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
            continue
        
        try:
            lots_index, lots_components = get_lots_for_dispersion(LongShort.Short, index, timestamp, True, False, False, False, notional_vega)
            # print(f"Index Lots: {lots_index} || Components Lots: {lots_index, lots_components}")

            if lots_index == 0:
                print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
                # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
                continue

            take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', index, LongShort.Short, lots_index)
            for (component, lots_component) in zip(index.components.values(), lots_components):
                take_dispersion_position((timestamp, time_ix), 'Long Dispersion Signal (zscore high, short index, long components)', component, LongShort.Long, lots_component)
            PTSL.fresh_trade(timestamp)
            current_position = LongShort.Long
            trade_start_date = timestamp
            print(f"Long dispersion (Short Index Long Components) trade executed")
        except Exception as e:
            print(f"Couldn't Enter a Long Dispersion Position: {e}")  

        continue
    #######################################################################################################################################################


    #######################################################################################################################################################
    elif exit_signal(timestamp):
        print("************  SHORT DISPERSION SIGNAL ************  ")
        UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, False, False)
        if isTodayAnyExpiry(timestamp, index):
            print("Expiry Day today. Trading on Expiry Day is not Allowed")
            continue

        if not PTSL.is_valid(timestamp):
            print(f"{PTSL.nature} was Triggered at {PTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
            # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
            continue

        try:
            lots_index, lots_components = get_lots_for_dispersion(LongShort.Long, index, timestamp, True, False, False, False, notional_vega)
            # print(f"lots_index, lots_components: {lots_index, lots_components}")

            if lots_index == 0:
                print("Cant take Dispersion Position, lots index for target notional vega can't be 0")
                # UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
                continue
            
            take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', index, LongShort.Long, lots_index)
            for (component, lots_component) in zip(index.components.values(), lots_components):
                take_dispersion_position((timestamp, time_ix), 'Short Dispersion Signal (zscore low, long index, short components)', component, LongShort.Short, lots_component)
            PTSL.fresh_trade(timestamp)
            current_position = LongShort.Short
            trade_start_date = timestamp
            print(f"Short dispersion (Long Index Short Components) trade")
        except Exception as e:
            print(f"Couldn't Enter a Short Dispersion Position: {e}")  
        continue
    #######################################################################################################################################################

    UpdateDispersionTickers(time_ix, timestamp, "NoHedging", False, True, False)
    print("========================================================================================================================================================================")
    

file.close()
sys.stdout = original_stdout.value

In [ ]:
# visualise_dispersion_trade('2023-08-11', '2023-09-11', 'try.html')

In [95]:
result_df = visualise_dispersion_trade(
    start=index.df_futures.index[0], 
    end=index.df_futures.index[-1], 
    file_name='PLOT_Complete_Period_Summary.html'
)

In [96]:
fig_pnl = Plot.plot_df(result_df, 'Running PNL', 'Running PNL (without expenses)', 'Cumulative Expenses')
Plot.save_plot(fig_pnl, f'PLOT_PNL_for_{strategy_desc}.html')

In [97]:
fig_greeks = Plot.plot_df(result_df, 'Net Delta (L)', 'Net Vega')
Plot.save_plot(fig_greeks, f'PLOT_GREEKS_for_{strategy_desc}.html')

In [98]:
fig_signals = Plot.plot_df(result_df, 'Z-Score', 'IC', 'Active Position?')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_short, 0, len(index.df_futures), 'white')
fig_signals = Plot.draw_horizontal_line(fig_signals, zscore_threshold_long, 0, len(index.df_futures), 'white')
Plot.save_plot(fig_signals, f'PLOT_SIGNALS_for_{strategy_desc}.html')

In [99]:
arr = []
arr += [{**trade, 'symbol': index.symbol} for trade in index.Trades.tradesArr]
for comp in index.components.values():
    arr += [{**trade, 'symbol': comp.symbol} for trade in comp.Trades.tradesArr]

In [89]:
df_trades = pd.DataFrame(arr)
df_trades = df_trades.sort_values(by='timestamp')
df_trades.reset_index(inplace=True)
df_trades.drop(columns=['index'], inplace=True)

In [ ]:
Plot.save_df_to_excel(df_trades, f'INFO_Trades_for_{strategy_desc}.xlsx')

In [90]:
# df_trades.to_excel(f"INFO_Trades_for_{strategy_desc}.xlsx", index=False)
# Plot.save_excel_trades(f"INFO_Trades_for_{strategy_desc}.xlsx")

In [76]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()

In [77]:
metrics(100000)